## Demo: Run Multiple Intent Experiments

In [1]:
!rm -rf ./test

In [2]:
%%capture
from leaderboard.nlu.base import base_experiment, intent_experiment
import copy

### Chose Configurations

In [3]:
base_config = intent_experiment.IntentExperimentConfiguration(
    exclusion_percentage=0.,
    drop_intents_with_less_than=2,
    data=base_experiment.DataConfig(
        name="examples-rules-nlu",
        data_path="../../../examples/rules/data/nlu.yml",
        domain_path="../../../examples/rules/domain.yml",
    ),
    model=base_experiment.ModelConfig(),
    clear_rasa_cache=True,
)

In [4]:
variations = {
    "interesting-config-name1" : "./config2.yml" ,
    "another-very-interesting-config-because-reasons" : "./config1.yml" ,
}

In [5]:
configs = []
for name, config_path in variations.items():
    config = copy.deepcopy(base_config)
    config.model.name = name
    config.model.config_path = config_path
    configs.append(config)

In [6]:
configs

[IntentExperimentConfiguration(description='Basic Intent Experiment', model=ModelConfig(name='interesting-config-name1', config_path='./config2.yml'), data=DataConfig(name='examples-rules-nlu', domain_path='../../../examples/rules/domain.yml', data_path='../../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, drop_intents_with_less_than=2, exclusion_percentage=0.0),
 IntentExperimentConfiguration(description='Basic Intent Experiment', model=ModelConfig(name='another-very-interesting-config-because-reasons', config_path='./config1.yml'), data=DataConfig(name='examples-rules-nlu', domain_path='../../../examples/rules/domain.yml', data_path='../../../examples/rules/data/nlu.yml'), clear_rasa_cache=True, drop_intents_with_less_than=2, exclusion_percentage=0.0)]

### Run!

In [7]:
!ls -lh ./

total 48
-rw-r--r--  1 kabu  staff    12K Apr  1 16:39 Run-Multiple-Intent-Experiments.ipynb
-rw-r--r--  1 kabu  staff   213B Mar 31 17:41 config1.yml
-rw-r--r--  1 kabu  staff   176B Apr  1 14:33 config2.yml


In [8]:
intent_experiment.multirun(configs=configs, out_dir='./test')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:55<00:00, 57.59s/it]


### Inspect the Results

In [9]:
!ls -lh ./test/intent_experiment__examples-rules-nlu

total 0
drwxr-xr-x  7 kabu  staff   224B Apr  1 16:42 config:another-very-interesting-config-because-reasons,drop:2,exclude:0.0__2022-04-01_16-41-16
drwxr-xr-x  7 kabu  staff   224B Apr  1 16:41 config:interesting-config-name1,drop:2,exclude:0.0__2022-04-01_16-40-18


In [10]:
import glob
example = list(glob.glob("./test/intent_experiment__examples-rules-nlu/*"))[0]

In [11]:
example

'./test/intent_experiment__examples-rules-nlu/config:another-very-interesting-config-because-reasons,drop:2,exclude:0.0__2022-04-01_16-41-16'

In [12]:
!ls -lha {example}

total 8
drwxr-xr-x  7 kabu  staff   224B Apr  1 16:42 .
drwxr-xr-x  4 kabu  staff   128B Apr  1 16:41 ..
drwxr-xr-x  5 kabu  staff   160B Apr  1 16:41 .hydra
drwxr-xr-x  4 kabu  staff   128B Apr  1 16:41 data
-rw-r--r--  1 kabu  staff   2.9K Apr  1 16:42 intent_experiment.log
drwxr-xr-x  3 kabu  staff    96B Apr  1 16:41 model
drwxr-xr-x  7 kabu  staff   224B Apr  1 16:42 report


In [13]:
!head -n 5 {example}/intent_experiment.log

[2022-04-01 16:41:16,462][leaderboard.nlu.base.base_experiment][INFO] - Running Experiment IntentExperiment
[2022-04-01 16:41:16,463][leaderboard.nlu.base.base_experiment][INFO] - Description: Experiment where we 1. exclude all intents with less than a fixed number of examples and 2. during training also remove a fixed percentage of the training examples before training.
[2022-04-01 16:41:17,365][rasa.engine.training.hooks][INFO] - Starting to train component 'LexicalSyntacticFeaturizer'.
[2022-04-01 16:41:17,415][rasa.engine.training.hooks][INFO] - Finished training component 'LexicalSyntacticFeaturizer'.
[2022-04-01 16:41:17,540][rasa.engine.training.hooks][INFO] - Starting to train component 'CountVectorsFeaturizer'.


In [14]:
!tail -n 5 {example}/intent_experiment.log

 [ 0  0  2  0  1 14  5  2  2]
 [ 0  0  0  1  0  2  1  0  0]
 [ 0  0  0  1  0  1  1  0  1]
 [ 0  0  0  0  0  2  1  0  2]]
[2022-04-01 16:42:08,476][rasa.nlu.test][INFO] - Entity evaluation results:


In [15]:
!cat {example}/.hydra/config.yaml

description: Basic Intent Experiment
model:
  name: another-very-interesting-config-because-reasons
  config_path: ./config1.yml
data:
  name: examples-rules-nlu
  domain_path: ../../../examples/rules/domain.yml
  data_path: ../../../examples/rules/data/nlu.yml
clear_rasa_cache: true
drop_intents_with_less_than: 2
exclusion_percentage: 0.0


In [16]:
!head -n 5 {example}/.hydra/hydra.yaml
!echo '...'

hydra:
  run:
    dir: ./test/intent_experiment__${data.name}/config:${model.name},drop:${drop_intents_with_less_than},exclude:${exclusion_percentage}__${now:%Y-%m-%d_%H-%M-%S}
  sweep:
    dir: multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}
...


In [17]:
!ls {example}/report/

intent_confusion_matrix.png  intent_report.json
intent_errors.json           training_metadata__times.csv
intent_histogram.png


In [18]:
!cat {example}/report/training_metadata__times.csv

name, start_time, duration(min), duration(sec)
train_LexicalSyntacticFeaturizer1, 2022-04-01 16:41:17, 0.0003, 0.01891493797302246
train_CountVectorsFeaturizer2, 2022-04-01 16:41:17, 0.0002, 0.010974884033203125
train_DIETClassifier3, 2022-04-01 16:41:17, 0.3856, 23.13459300994873
